# Mean-Shift Clustering and Segmentation

In the first part of this task you will implement the *mean-shift* clustering algorithm in a general way (not specifically for anything to do with images, just simply for n-dimensional data points).

Then in the second part you will apply mean-shift for image segmentation, by clustering data points that represent pixels (e.g. the colors).

### Recap from the lecture
The *mean-shift* algorithm clusters an $n$-dimensional data set (i.e., each data point is described by a feature vector of $n$ values) by associating each point with a peak in the estimated probability density of the dataset's distribution. Points associated with the "same" peak (up to some small threshold) become members of the same cluster.

For each point, mean-shift finds the associated peak by first defining a spherical window of radius $r$ centered on that point, and computing the **mean** of the points that lie within the window. The algorithm then **shifts** the window to be centered on that mean and repeats these steps until convergence, i.e., until the shift is smaller than a specified threshold. At each iteration the window shifts to a more densely populated portion of the data set until a peak is reached, where the data is approximately equally distributed in the window.

*Important*: Implement the following tasks efficiently, otherwise your code might need hours to run. Avoid loops, instead use numpy-functions when possible.

In [1]:
%%html
<!-- Run this cell to add heading letters per subtask (like a, b, c) -->
<style>
body {counter-reset: section;}
h2:before {counter-increment: section;
           content: counter(section, lower-alpha) ") ";}
</style>

In [2]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
import time
import imageio
from mpl_toolkits.mplot3d import Axes3D
import cv2
import os

## Finding a peak from a query point
Implement the peak searching process as the function `find_peak(data, query, radius)` where

 * `data` is a $p \times n$ matrix containing $p$ data points; each point is defined by an $n$-dimensional row vector of feature values
 * `query` is the $n$-dimensional starting point from which we wish to compute a density peak
 * `radius` is the search window radius.

Return the peak as an $n$-dimensional vector once the peak does not change anymore.

**Hints:**
* Only a single loop (shift while peak is moving) is required in this task.
* `np.linalg.norm` computes the Euclidean norm of a vector. You can also apply it to a matrix (which represents a list of vectors) by specifying the `axis` parameter.
* NumPy arrays can be indexed with Boolean arrays, e.g. to select only rows that fulfil some criterion.

In [3]:
# POINTS: 2

def find_peak(data, query, radius):
    # YOUR CODE HERE    
    thresh=0.000005
    diff=1
    while diff>thresh:
        #calc euclidean dist of every point[Hypersphere dist]
        rad_mat = np.linalg.norm(query-data,axis=1)
        #cut of w.r.t radius 
        truth_mat=np.full((data.shape[0]), False)
        truth_mat[rad_mat<=radius]=True
        #calculate mean of points that are within radius
        pres_mean=np.mean(data[truth_mat==True],axis=0)
        #approximate new index w.r.t mean value
        prev_mean=query
        query=pres_mean
        diff=np.linalg.norm(query-prev_mean)

    
    peak_vec=pres_mean
    return peak_vec
    
    raise NotImplementedError()

We will now use the synthetic dataset `gaussian_mixture_samples_3d.csv` to test your implementation. The data points in this file are 2000 samples from two 3D Gaussian distributions. The following plots only show the projection on the XY plane.

In [4]:
data = np.genfromtxt(f'gaussian_mixture_samples_3d.csv', delimiter=',')
query_ids = [0, 5, 1500]
radius = 2

fig, axes = plt.subplots(1, len(query_ids), figsize=(9.5,3.5))
for query_id, ax in zip(query_ids, axes):
    query = data[query_id]
    peak = find_peak(data, query, radius)
    print('Found peak', peak)
    
    ax.scatter(data[:, 0], data[:, 1], marker='.', color='gray')
    ax.scatter(query[0], query[1], s=150, linewidth=5,
               color='blue', marker='x', label='starting point')
    ax.scatter(peak[0], peak[1], color='orange', marker='x',
               s=150, linewidth=5, label='found peak')
    ax.legend()
fig.tight_layout()

<IPython.core.display.Javascript object>

Found peak [-0.06218913  0.05139093  0.03810532]
Found peak [-0.05623617  0.05895276  0.03974915]
Found peak [5.04059462 4.98140242 5.0179125 ]


## Clustering all points
Implement `mean_shift(data, radius)`, which calls `find_peak` for each point and then assigns a label to each point according to its peak.
`mean_shift` should return two arrays: `peaks` and `labels`.

  * `peaks` is a matrix with $k$ rows and $n$ columns, storing the found density peaks of size $n$, where $k$ is the data-dependent number of clusters found. 
  * `labels` is a $p$-sized vector that has an entry for each data point, storing its associated cluster label (an integer).

Similar peaks within a distance of `radius/2` should be considered the same and should be merged after each call to `find_peak`. More specifically, if the peak computed for a data point already exists in `peaks` (according to the distance threshold), then discard the newly computed peak and give the label of the already existing peak to the considered data point.

**Hints:**
* Only a single loop (iterating over each point once) is required in this task. (Plus the loop within `find_peak`.)
* `np.argmin` gives the index of the smallest value of a vector.

In [5]:
# POINTS: 4

def mean_shift(data, radius):
    labels = np.full(len(data), fill_value=-1, dtype=int)
   
    # YOUR CODE HERE
    #peaks=np.array(data.shape[1])
    peaks=[]
    pt_lbl=0
 
    for i,dp in enumerate(data):
        peak_val=find_peak(data, dp, radius)

        if len(peaks)==0:
            peaks.append(peak_val)
            labels[i]=pt_lbl

            continue
        idx=np.argmin(np.linalg.norm(peak_val-peaks,axis=1)) #Gives closest mode peak ideally
        comp_vec=peaks[idx]
        mag_val=np.linalg.norm(peak_val-comp_vec) 
        # check if it falls within radius/2  if it does discard and assign same label
        if(mag_val<radius/2)and(i!=0):
            labels[i]=idx

        elif np.isin(peak_val,peaks).all() ==False:
            peaks.append(peak_val)
            pt_lbl+=1       
            labels[i]=pt_lbl 


    return np.array(peaks), labels

Now check the result of your implementation. The found peaks (cluster centers) are shown as black X marks. You can rotate the interactive 3D plots with the mouse (but it may be somewhat slow).

Note that each individual clustering should take less than 10 seconds to compute.

In [6]:
def plot_3d_clusters(ax, data, labels, peaks, 
                     peak_colors=None, colors=None, axis_names='xyz'):
    """Plots a set of point clusters in 3D, each with different color."""

    def luv2rgb(color):
        expanded = color[np.newaxis, np.newaxis]
        rgb = cv2.cvtColor(expanded.astype(np.uint8), cv2.COLOR_LUV2RGB)
        return rgb[0,0]/255
      
    if peak_colors is None:
        peak_colors = peaks
    
    for label in range(len(peaks)):
        if colors=='rgb':
            cluster_color = color = peak_colors[label]/255
        elif colors=='luv':
            cluster_color = luv2rgb(peak_colors[label])
        else:
            cluster_color=None

        cluster = data[labels==label]
        ax.scatter(cluster[:, 0], cluster[:, 1], cluster[:, 2],
                   alpha=0.15, color=cluster_color)
        ax.scatter(peaks[label, 0], peaks[label, 1], peaks[label, 2], 
                   color='black', marker='x', s=150, linewidth=3)

    ax.set_xlabel(axis_names[0])
    ax.set_ylabel(axis_names[1])
    ax.set_zlabel(axis_names[2])

data = np.genfromtxt(f'gaussian_mixture_samples_3d.csv', delimiter=',')
radii = [1, 1.25, 2, 8]
fig, axes = plt.subplots(
    1, len(radii), figsize=(9.5,4), subplot_kw={'projection': '3d'})

for radius, ax in zip(radii, axes): 
    start_time = time.time()
    peaks, labels = mean_shift(data, radius)
    plot_3d_clusters(ax, data, labels, peaks)
    duration = time.time()-start_time
    ax.set_title(f'{len(peaks)} peaks using r={radius:.2f}\n'
                 f'Needed {duration:.4f}s\n')
    
fig.tight_layout()

<IPython.core.display.Javascript object>

## Speedups

As described so far, the mean-shift algorithm is too slow to be used for image segmentation where each data point corresponds to a pixel, there are just too many pixels in normal sized images. Therefore, you should incorporate the following two speedups from the lecture into your implementation. 

**First speedup**: upon finding a new peak, associate each data point within `radius` distance from that peak with the cluster defined by that peak. This speedup is known as the *“basin of attraction”* and is based on the intuition that points within one window size distance from the peak will, with high probability, converge to that peak.

Call the new function `mean_shift_opt`.


**Hint:** You can use boolean indexing to assign a value only to certain array elements.
```python
>>> a = np.array([1, 3, 2, 4])
>>> b = a > 2
>>> b
np.array([False, True, False, True])
>>> a[b] = 0
>>> a
array([1, 0, 2, 0])
```

In [7]:
# POINTS: 2

def mean_shift_opt(data, radius):
    labels = np.full(len(data), fill_value=-1, dtype=int)
    t_arr=np.full(len(data), fill_value=False, dtype=bool)
    # YOUR CODE HERE
    peaks=[]
    labeled=set()
    pt_lbl=0
    i=0
    for k,dp in enumerate(data):
        peak_val=find_peak(data, dp, radius)
        
        if len(peaks)==0:
            peaks.append(peak_val)
            labels[k]=pt_lbl
            t_arr=(np.linalg.norm(peak_val-data,axis=1)<=radius)
         
            labels[t_arr] = pt_lbl
            pt_lbl+=1
   
            continue
       
        if labels[k]!=-1 and (labels[k] in labeled==True):
            continue
            

   
    
        
        idx=np.argmin(np.linalg.norm(peak_val-peaks,axis=1)) #Gives closest mode peak ideally
        comp_vec=peaks[idx]
        mag_val=np.linalg.norm(peak_val-comp_vec) 
        # check if it falls within radius/2  if it does discard and assign same label
        if(mag_val<radius/2)and(k!=0):
            labels[k]=idx

        elif np.isin(peak_val,peaks).all() ==False:
            peaks.append(peak_val)
            pt_lbl+=1       

            t_arr=(np.linalg.norm(peak_val-data,axis=1)<=radius)
         
            labels[t_arr] = pt_lbl
        labeled.add(pt_lbl)

    return np.array(peaks), labels

Now run the code to check the result.

In [8]:
data = np.genfromtxt(f'gaussian_mixture_samples_3d.csv', delimiter=',')
radii = [1, 1.25, 2, 8]
fig, axes = plt.subplots(
    1, len(radii), figsize=(9.5,4), subplot_kw={'projection': '3d'})

for radius, ax in zip(radii, axes): 
    start_time = time.time()
    peaks, labels = mean_shift_opt(data, radius)
    plot_3d_clusters(ax, data, labels, peaks)
    duration = time.time()-start_time
    ax.set_title(f'{len(peaks)} peaks using r={radius:.2f}\n'
                 f'Needed {duration:.4f}s\n')
    
fig.tight_layout()

<IPython.core.display.Javascript object>

The **second speedup** is based on a similar principle: Associate points within a distance $\mathtt{radius}/c$ of the search path with the converged peak ($c$ is some constant value). Use $c = 3$ for this assignment.

To realize this speedup, you will need to modify `find_peak` into `find_peak_opt`, which returns two values: `peak` and `is_near_search_path`. The latter should be a Boolean output vector of length $p$ (number of data points) containing `True` for each point that we encountered within a distance $\mathtt{radius}/c$ on our search path, and `False` for the others. Then use this Boolean vector in a new version of `mean_shift_opt`, called `mean_shift_opt2` to do the label assignments accordingly.

In [ ]:
# POINTS: 2

def find_peak_opt(data, query, radius, c=3):
    is_near_search_path = np.zeros(len(data), dtype=bool)
    
    # YOUR CODE HERE
    thresh=0.000005
    diff=1
    while diff>thresh:
        #calc euclidean dist of every point[Hypersphere dist]
        rad_mat = np.linalg.norm(query-data,axis=1)
        #cut of w.r.t radius 
        truth_mat=np.full((data.shape[0]), False)
        truth_mat[rad_mat<=radius]=True
        #calculate mean of points that are within radius
        pres_mean=np.mean(data[truth_mat==True],axis=0)
        #approximate new index w.r.t mean value
        prev_mean=query
        query=pres_mean
        diff=np.linalg.norm(query-prev_mean)
        is_near_search_path[diff <= radius/c] = True
    
    peak_vec=pres_mean
    return peak_vec,is_near_search_path

def mean_shift_opt2(data, radius):
   
    # YOUR CODE HERE
    labels = np.full(len(data), fill_value=-1, dtype=int)
    t_arr=np.full(len(data), fill_value=False, dtype=bool)
    peaks=[]
    labeled=set()
    pt_lbl=0
    i=0
    for k,dp in enumerate(data):
        peak_val, is_near_search_path =find_peak_opt(data, dp, radius)
        
        if len(peaks)==0:
            peaks.append(peak_val)
            labels[k]=pt_lbl
            t_arr=(np.linalg.norm(peak_val-data,axis=1)<=radius)
         
            labels[t_arr] = pt_lbl
            pt_lbl+=1
            continue
       
        if labels[k]!=-1 and (labels[k] in labeled==True):
            continue
            

   
    
        
        idx=np.argmin(np.linalg.norm(peak_val-peaks,axis=1)) #Gives closest mode peak ideally
        comp_vec=peaks[idx]
        mag_val=np.linalg.norm(peak_val-comp_vec) 
        # check if it falls within radius/2  if it does discard and assign same label
        if(mag_val<radius/2)and(k!=0):
            labels[k]=idx

        elif np.isin(peak_val,peaks).all() ==False:
            peaks.append(peak_val)
            pt_lbl+=1        
            t_arr=(np.linalg.norm(peak_val-data,axis=1)<=radius)
         
            labels[t_arr] = pt_lbl
        labeled.add(pt_lbl)
        labels[is_near_search_path] = pt_lbl
        
    return np.array(peaks), labels

data = np.genfromtxt(f'gaussian_mixture_samples_3d.csv', delimiter=',')
radii = [1, 1.25, 2, 8]
fig, axes = plt.subplots(
    1, len(radii), figsize=(9.5,4), subplot_kw={'projection': '3d'})

for radius, ax in zip(radii, axes):
    start_time = time.time()
    peaks, labels = mean_shift_opt2(data, radius)
    plot_3d_clusters(ax, data, labels, peaks)
    duration = time.time()-start_time
    ax.set_title(f'{len(peaks)} peaks using r={radius:.2f}\n'
                 f'Needed {duration:.4f}s\n')
    
fig.tight_layout()

<IPython.core.display.Javascript object>

Which radius gives good results? Can the speedups change the result? Why is the speedup so different for the different radii?

*POINTS: 0*

YOUR ANSWER HERE

## Image Segmentation by Mean-Shift

Now use your mean-shift implementation from above to segment images. Note that although very efficient mean-shift implementations exist, our version here may take several minutes per image. Debug using small images.

Implement the function `mean_shift_segment(im, radius)` where `im` is an input RGB image of shape $\mathtt{height}\times\mathtt{width}\times 3$ and `radius` is the parameter associated with the mean-shift algorithm. The output should be `data`, `peaks`, `labels`, `segmented_image`:

* `data` is the array of the data points that you input to the mean-shift algorithm, with $p$ rows and 3 columns (where $p=\mathtt{height}\cdot\mathtt{width}$).
* `peaks` and `labels` are simply the results returned by the `mean_shift` call (without changing them).
* The `segmented_image` is constructed by assigning to each pixel the color value of the corresponding cluster's peak.

You will need to reshape (`np.reshape`) the image array before feeding it to your mean-shift clustering function. Also, do not forget to convert the pixel values to floating point, using `somearray.astype(float)` and then convert the result back to 8-bit unsigned integers using `somearray.astype(np.uint8)`.

**Hints:** You can change the shape of an array `a` with `a.reshape()`.

In [ ]:
# POINTS: 2

def mean_shift_segment(im, radius):
    # YOUR CODE HERE
    data = im.reshape(-1, 3)
    peaks, labels = mean_shift(data, radius)
    
    segmented_im = peaks[labels]
    segmented_im=segmented_im.reshape(im.shape).astype(np.uint8)
   

    return data, peaks, labels, segmented_im

The next cell segments the image `terrain_small.png`. It should take less than 5 seconds.

In [ ]:
def make_label_colormap():
    """Create a color map for visualizing the labels themselves,
    such that the segment boundaries become more visible, unlike
    in the visualization using the cluster peak colors.
    """
    import matplotlib.colors
    rng = np.random.RandomState(2)
    values = np.linspace(0, 1, 20)
    colors = plt.cm.get_cmap('hsv')(values)
    rng.shuffle(colors)
    return matplotlib.colors.ListedColormap(colors)

label_cmap = make_label_colormap()

im = imageio.imread('terrain_small.png')
start_time = time.time()
data, peaks, labels, segmented_im = mean_shift_segment(im, radius=15)
duration= time.time()-start_time
print(f'Took {duration:.2f} s')

fig = plt.figure(figsize=(9.5,9.5))
ax = fig.add_subplot(2, 2, 1)
ax.set_title('Original Image')
ax.imshow(im)

ax = fig.add_subplot(2, 2, 2)
ax.set_title('Segmented')
ax.imshow(segmented_im)

ax = fig.add_subplot(2, 2, 3)
ax.set_title('Labels')
ax.imshow(labels.reshape(im.shape[:2]), cmap=label_cmap)

ax = fig.add_subplot(2, 2, 4, projection='3d')
ax.set_title(f'RGB space')
plot_3d_clusters(ax, data, labels, peaks, colors='rgb', axis_names='RGB')
fig.tight_layout()

## Segmentation in LUV color space
Note that mean-shift uses the Euclidean distance metric. Unfortunately, the Euclidean distance in RGB color space does not correlate well to color difference as perceived by people. For example in the green portion of RGB space, large distances are perceived as the roughly same color, whereas in the blue part a small RGB-distance may cause a large change in perceived color. For this reason we will now use the non-linear LUV color space, where the Euclidean distance better models the perceived difference in color.

In the function `mean_shift_segment_luv(...)` cluster the image data in LUV color space by first converting the RGB color vectors to LUV using OpenCV: `cv2.cvtColor(rgb_image, cv2.COLOR_RGB2LUV)`. Then convert the resulting cluster centers back to RGB using `cv2.cvtColor(luv_image, cv2.COLOR_LUV2RGB)`.

If we want to include spatial *position information* in the feature vectors as well, we can make the feature vectors 5 dimensional by specifying in addition to the the L, U, and V values also the x, and y coordinates of each pixel. Write a function `mean_shift_segment_luv_pos(im, radius)` that does this. The parameter `pos_weight` should be multiplied to the position coordinates.

**Hint:** useful functions are `np.arange`, `np.meshgrid`, `np.concatenate`, `np.expand_dims`.

In [ ]:
# POINTS: 2

def mean_shift_segment_luv(im, radius):
    # YOUR CODE HERE
    luv_img = cv2.cvtColor(im, cv2.COLOR_RGB2LUV)
    im_data = luv_img.reshape(-1, 3)
    
    peaks, labels = mean_shift(im_data, radius)
    
    segmented_im = peaks[labels]
    segmented_im = segmented_im.reshape(im.shape).astype(np.uint8)
    segmented_im = cv2.cvtColor(segmented_im, cv2.COLOR_LUV2RGB)
    
    return data, peaks, labels, segmented_im
    
def mean_shift_segment_luv_pos(im, radius, pos_weight=1):
    # YOUR CODE HERE
    luv_img = cv2.cvtColor(im, cv2.COLOR_RGB2LUV)
    im_data = luv_img.reshape(-1, 3)
    
    Xshape = np.arange(im.shape[1])
    Yshape = np.arange(im.shape[0])
#     Xshape = np.linspace(0, im.shape[1], 1)
#     Yshape = np.linspace(0, im.shape[0], 1)
    xcords, ycords = np.meshgrid(Xshape,Yshape)
    xy_combim = np.stack([xcords, ycords], axis=-1)
    
    spatial_img = np.concatenate([luv_img, pos_weight*xy_combim], axis=-1)
    data = spatial_img.reshape(-1, 5).astype(float)
    
    peaks, labels = mean_shift(im_data, radius)
    
    segmented_im = peaks[labels]
    segmented_im = segmented_im.reshape(im.shape).astype(np.uint8)
    segmented_im = cv2.cvtColor(segmented_im, cv2.COLOR_LUV2RGB)
    
    
#     raise NotImplementedError()
    return data, peaks, labels, segmented_im

In [ ]:
im = imageio.imread('terrain_small.png')
data, peaks, labels, segmented_im = mean_shift_segment_luv(im, radius=10)
fig = plt.figure(figsize=(9.5,11))

ax = fig.add_subplot(4, 3, 1)
ax.set_title('Segmented LUV')
ax.imshow(segmented_im)

ax = fig.add_subplot(4, 3, 2)
ax.set_title('Labels LUV')
ax.imshow(labels.reshape(im.shape[:2]), cmap=label_cmap)

ax = fig.add_subplot(4, 3, 3, projection='3d')
ax.set_title(f'LUV space')
plot_3d_clusters(ax, data, labels, peaks, colors='luv', axis_names='LUV')

for i, pos_weight in enumerate([0.5, 1, 2]):
    data, peaks, labels, segmented_im = mean_shift_segment_luv_pos(im, radius=15, pos_weight=pos_weight)
    ax = fig.add_subplot(4, 3, 4 + 3*i)
    ax.set_title(f'Segmented LUV + pos (weight {pos_weight})')
    ax.imshow(segmented_im)

    ax = fig.add_subplot(4, 3, 5 + 3*i)
    ax.set_title(f'Labels LUV + pos (weight {pos_weight})')
    ax.imshow(labels.reshape(im.shape[:2]), cmap=label_cmap)

    ax = fig.add_subplot(4, 3, 6 + 3*i, projection='3d')
    ax.set_title(f'VXY space')
    plot_3d_clusters(
        ax, data[:, 2:], labels, peaks[:, 2:], 
        peak_colors=peaks[:, :3], colors='luv', axis_names='VXY')
    ax.invert_zaxis()
    ax.view_init(azim=20, elev=15)

    fig.tight_layout()

What effect does adding position information have? Why does it make sense to use a bigger radius for more dimensions ($15$ instead of $10$ in the example above)? What is the effect of weighting the positions?

*POINTS: 0*

YOUR ANSWER HERE

## Experiment with the parameters

Run the next cell. How does the `radius` and the choice of the feature vector affect the resulting segmentations?

*POINTS: 0*

YOUR ANSWER HERE

In [ ]:
im = imageio.imread('terrain_small.png')
radii = [5, 10, 20]
fig, axes = plt.subplots(len(radii), 3, figsize=(9.5,7.5))
for radius, ax in zip(radii, axes):
    segmented_im = mean_shift_segment(im, radius)[-1]
    ax[0].imshow(segmented_im)
    ax[0].set_title(f'Radius {radius} RGB')
    
    segmented_im = mean_shift_segment_luv(im, radius)[-1]
    ax[1].imshow(segmented_im)
    ax[1].set_title(f'Radius {radius} LUV')

    segmented_im = mean_shift_segment_luv_pos(im, radius)[-1]
    ax[2].imshow(segmented_im)
    ax[2].set_title(f'Radius {radius} LUV+pos')
fig.tight_layout()

## [OPTIONAL] Test it on a larger image

Run your algorithm on at least one larger (approx. 250x250) test image using your own choice of `radius` and feature vector definition. One source for possible test images is the dataset of images available at http://www.eecs.berkeley.edu/Research/Projects/CS/vision/grouping/segbench/. You can also try the example images included in the scikit-image library, e.g. `import skimage.data; im = skimage.data.astronaut()`. Or any image from anywhere.

Processing can take several minutes for larger images.

In [ ]:
# POINTS: 0

import skimage.data
im = skimage.data.astronaut()
im = cv2.resize(im, (250,250))

# YOUR CODE HERE
raise NotImplementedError()

fig, axes = plt.subplots(1, 3, figsize=(9.5, 4))
axes[0].set_title('Original image')
axes[0].imshow(im)
axes[1].set_title('Segmented image')
axes[1].imshow(segmented_im)
axes[2].set_title('Labels')
axes[2].imshow(labels.reshape(im.shape[:2]), cmap=label_cmap)
fig.tight_layout()